In [71]:
from tensorflow.examples.tutorials.mnist import input_data

In [72]:
import numpy as np
from __future__ import print_function
import tensorflow as tf

In [96]:
class Network():

    alpha = 0.1

    def __init__(self, learning_rate=0.1):
        ''' initialize the classifier with default (best) parameters '''
        # TODO
        self.alpha = learning_rate

        # Parameters
        learning_rate = 0.001
        self.batch_size = 30
        self.display_step = 1

        # Network Parameters
        n_hidden_1 = 256 # 1st layer number of neurons
        n_hidden_2 = 256 # 2nd layer number of neurons
        n_input = 784 # MNIST data input (img shape: 28*28)
        n_classes = 10 # Classes (0-9 digits)

        # tf Graph input
        self.X = tf.placeholder("float", [None, n_input])
        self.Y = tf.placeholder("float", [None, n_classes])

        # Store layers weight & bias
        weights = {
            'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
            'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
            'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
        }
        biases = {
            'b1': tf.Variable(tf.random_normal([n_hidden_1])),
            'b2': tf.Variable(tf.random_normal([n_hidden_2])),
            'out': tf.Variable(tf.random_normal([n_classes]))
        }

        # Hidden fully connected layer with 256 neurons
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(self.X, weights['h1']), biases['b1']))
        # Hidden fully connected layer with 256 neurons
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
        # Output fully connected layer with a neuron for each class
        self.logits = tf.matmul(layer_2, weights['out']) + biases['out']

        # Define loss and optimizer
        self.loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.Y))
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        self.train_op = optimizer.minimize(self.loss_op)

        # Initializing the variables
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)

    def fit(self,X,Y,warm_start=True,n_epochs=100):
        ''' train the network, and if warm_start, then do not reinit. the network
            (if it has already been initialized)
        '''
        #self.n_labels = Y.shape[1]

        # Training cycle
        for epoch in range(n_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/self.batch_size)
            # Loop over all batches
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(self.batch_size)
                # Run optimization op (backprop) and cost op (to get loss value)
                _, c = self.sess.run([self.train_op, self.loss_op], feed_dict={self.X: batch_x, self.Y: batch_y})
                # Compute average loss
                avg_cost += c / total_batch
            # Display logs per epoch step
            if epoch % self.display_step == 0:
                print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
        print("Optimization Finished!")

    def predict_proba(self,X):
        ''' return a matrix P where P[i,j] = P(Y[i,j]=1), 
        for all instances i, and labels j. '''

        predictions = {
              # Generate predictions (for PREDICT and EVAL mode)
              "classes": tf.argmax(input=self.logits, axis=1),
              # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
              # `logging_hook`.
              "probabilities": tf.nn.softmax(self.logits, name="softmax_tensor")
        }

        return predictions["probabilities"].eval(feed_dict={self.X: mnist.test.images}, session=self.sess)

    def predict(self,X):
        ''' return a matrix of predictions for X '''
        return (self.predict_proba(X) >= 0.5).astype(int)

In [97]:
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [98]:
h = Network()

In [99]:
h.fit(1,2)

Epoch: 0001 cost=1.213979099
Epoch: 0002 cost=0.384473990
Epoch: 0003 cost=0.244094186
Epoch: 0004 cost=0.167919970
Epoch: 0005 cost=0.118508170
Epoch: 0006 cost=0.086843417
Epoch: 0007 cost=0.061206527
Epoch: 0008 cost=0.043666708
Epoch: 0009 cost=0.032730134
Epoch: 0010 cost=0.022780091
Epoch: 0011 cost=0.017355429
Epoch: 0012 cost=0.013035941
Epoch: 0013 cost=0.011634027
Epoch: 0014 cost=0.007786019
Epoch: 0015 cost=0.008720020
Epoch: 0016 cost=0.005252224
Epoch: 0017 cost=0.006047913
Epoch: 0018 cost=0.005879202
Epoch: 0019 cost=0.002722073
Epoch: 0020 cost=0.007413884
Epoch: 0021 cost=0.001331336
Epoch: 0022 cost=0.003593847
Epoch: 0023 cost=0.009132252
Epoch: 0024 cost=0.000970939
Epoch: 0025 cost=0.005696144
Epoch: 0026 cost=0.000728002
Epoch: 0027 cost=0.007818529
Epoch: 0028 cost=0.001954308
Epoch: 0029 cost=0.002480318
Epoch: 0030 cost=0.007295658
Epoch: 0031 cost=0.001339435
Epoch: 0032 cost=0.007395078
Epoch: 0033 cost=0.002552610
Epoch: 0034 cost=0.001219776
Epoch: 0035 co

In [100]:
proba = h.predict_proba(1)
proba[0]

array([3.0305915e-21, 1.4130888e-18, 7.1772158e-14, 9.2752598e-18,
       1.1935199e-22, 4.3875520e-22, 2.3295744e-27, 1.0000000e+00,
       3.3099278e-22, 1.1765401e-16], dtype=float32)

In [101]:
Y_pred = h.predict(1)
Y_pred

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [102]:
loss = np.mean(Y_pred != mnist.test.labels)
print("Hamming loss: ", loss)

Hamming loss:  0.00622
